# Andrianarivo RAKOTOARIJAONA:

Le travail réalisé ici a pour but la construction de base de données pour un projet fil rouge en text mining. 

## Scraper indeed avec beautifulSoup:

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import clear_output

## Création d'une fonction scrap:
La fonction **scrap_indeed_offer** a comme objectif de scraper des offres sur indeed. Il suffit d'ajouter l'url de l'offre sans le nom de la ville. Comme exemple, si l'on veut avoir des offres concernant le poste de data scientit on tape sur indeed data scientist avec comme ville Paris et cela nous donne comme URL:https://www.indeed.fr/jobs?q=data+scientist&l=Paris.
Dans notre cas, le paramètre URL va recevoir que https://www.indeed.fr/jobs?q=data+scientist. Cela est nécessaire car nous en veut récupérer les offres dans chaque ville que l'on soit en France, d'où le paramètre city_set. Dans notre cas, on va récuperer toutes

In [2]:
def scrap_indeed_offer(URL,city_set,nb_resu_par_ville=1,extension="https://www.indeed.com"):
    columns = ["ville", "utituler_offre","description","entreprise","lacalisation", "resume"]
    sample_df = pd.DataFrame(columns = columns)
    for city in city_set:
      for start in range(0,nb_resu_par_ville, 10):
          page = requests.get(URL+"&l="+str(city)+'&start='+str(start))
          soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
          for div in soup.find_all(name="div", attrs={"class":"row"}): 
            #specification du nombre de lignes
            num = (len(sample_df) + 1) 
            
            job_post = [] 
            job_post.append(city) 
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
              job_post.append(a["title"])
             # recuperer site sousjacent contenant le descriptif de l'offre 
              url_fils=extension+a["href"]
              page_fils=requests.get(url_fils)
              soup_fils=BeautifulSoup(page_fils.text,"lxml",from_encoding="utf-8")
              div_class="jobsearch-JobComponent-description icl-u-xs-mt--md"
              for desc in soup_fils.find_all(name="div",attrs={"class":div_class}):
                  if len(desc.text)>0:
                     job_post.append(desc.text)
                  else:
                     job_post.append("")
              #NOm de la companie
              company = div.find_all(name="span", attrs={"class":"company"}) 
              if len(company) > 0: 
                  for b in company:
                    job_post.append(b.text.strip()) 
              else: 
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                for span in sec_try:
                    job_post.append(span.text) 
        #localisation
              c = div.findAll("span", attrs={"class": "location"}) 
              if len(c)>0:
                for span in c: 
                    job_post.append(span.text)
              else:
                    job_post.append("pas localisé")
        #resumer
              d = div.findAll("span", attrs={"class": "summary"}) 
              for span in d:
                if len(span.text.strip())>0:
                    job_post.append(span.text.strip())
                else:
                    job_post.append("")        
        #mise a jours de notre dataframe
            clear_output(wait=True)
            print("nb_boocle:{}".format(num))
            print("len job_post={}".format(len(job_post)))
            print(job_post)
            if len(job_post)==sample_df.shape[1]:
                sample_df.loc[num,:] = job_post
    t=["ville", "utituler_offre","entreprise","localisation", "resume","description"]
    res=sample_df.loc[:,t]
    return(res)

## Scraper les offres en data science avec au plus 100 offres par ville:

In [4]:
nb_resu_par_ville = 30
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
URL="https://www.indeed.com/jobs?q=data+scientist"
t1=time.time()
res=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

nb_boocle:25270
len job_post=6
['Panama+City', 'SharePoint Consultant', 'JOB DESCRIPTIONProvide full time on-site advanced SharePoint consulting, development and support services for a Solution Foundry government customer in the Panama City, Fl area.MUST-HAVESThe ideal candidate possesses the capabilities and experience listed below and can demonstrate ability to expertly apply these skills in a SharePoint environment.3+ recent years of primary work experience as a Microsoft SharePoint power user and/or developer2+ years of experience supporting federal government customer(s)Ability to design, develop and implement advanced SharePoint 2013, 2016 or Office 365 solutionsAdvanced knowledge of all out-of-the-box SharePoint features, functionality and configurationsExperience managing SharePoint site content, sites/site collections, permissions and groups, and workflowsExperience customizing SharePoint sites using SharePoint DesignerAbility to analyze data and determine opportunities for Sh

/home/tantely/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


NameError: name 'delta' is not defined

In [5]:
res1=res.drop_duplicates()

In [6]:
res1.shape

(19253, 6)

L'execution de la fonction prend un certain temps pour **n_resu_par_ville=1000**. Mais comme on a pas de contrainte de temps cela ne pose pas de problème.

In [7]:
res1.to_csv("poste_data_scientist_scraped_usa.csv",index=False)

### UK scrap data science:

In [53]:
nb_resu_par_ville = 100
city_set = ["London","Birmingham","Leeds","Glasgow",
            "Sheffield","Bradford","Liverpool","Edinburgh",
           "Manchester","Bristol","Kirklees","Newcastle-upon-Tyne",
           "Kingston-upon-Hull","Bolton","Walsall","Plymouth",
           "Nottingham","Doncaster","Sunderland","Scottish+Borders",
           "Bassetlaw","Cheltenham","Erewash","East+Dunbartonshire",
           "Cambridgeshire","Norwich","Highland","Derby"]
URL="https://www.indeed.co.uk/jobs?q=Data+Scientist"
extension="https://www.indeed.co.uk"
res_uk=scrap_indeed_offer(URL,city_set,nb_resu_par_ville,extension)

nb_boocle:3663
len job_post=6
['Derby', 'Computational Scientist, Group Rios', 'The Princess Máxima Center for Pediatric Oncology is a new research hospital concentrating healthcare, research and education with regard to cancer in children in a single location in Utrecht. The institute aims to provide the highest level of care for all children with cancer and has the ambition to cure all children of cancer and significantly reduce unwanted side effects. The center brings together the best possible care and scientific research, creating a unique interdisciplinary institute for pediatric oncology in Europe.The research team led by Anne Rios, PhD has recently started to study the causes of childhood cancer and treatment-associated second malignancies via state-of-the-art-microscopy methods. By understanding the processes that cause cancers, effective preventive therapies can be developed, diagnostics improved and choice of treatment facilitated.To achieve these goals, we combine mouse gen

In [63]:
res_dedupliquer_uk=res_uk.drop_duplicates()

(1426, 6)

In [64]:
res_dedupliquer_uk.to_csv("data_science_job_uk.csv",index=False)

## Data scientist au canada:

In [74]:
nb_resu_par_ville = 100
city_set = ["Toronto","Montreal","Calgary","Ottawa",
            "Edmonton","Mississauga","Winnipeg","Vancouver",
           "Brampton","Hamilton","Surrey","Halifax",
            "Quebec+City","Markham","Gatineau","Oakville",
            "Delta","Terrebonne","Ajax","Lévis",
           "Oshawa","Windsor","Burnaby","Richmond"]
URL="https://ca.indeed.com/jobs?q=Data Scientist"
extension="https://ca.indeed.com"
res_canada=scrap_indeed_offer(URL,city_set,nb_resu_par_ville,extension)

nb_boocle:2199
len job_post=6
['Richmond', 'Human Resources Generalist', 'STEMCELL Technologies Inc. is a privately-owned biotechnology company based in Vancouver that helps power leading-edge life science research around the world. Scientists performing stem cell, immunology, cancer, regenerative medicine and cellular therapy research are among those who rely on our cell culture media, cell separation products, instruments, ancillary reagents and contract assay services. We create novel, useful, standardized products of unfailing quality and deliver them to more than 70 countries via our many regional offices plus distribution centres in Vancouver, Seattle, Grenoble and Singapore. Driven by our love of science and our passion for quality, we see ourselves simply as “Scientists Helping Scientists” – standing by our customers to provide outstanding products, technical support and training. We have over 1000 science-oriented employees globally, including 250 PhDs/MScs, with most others h

In [75]:
res_duplicate_ca=res_canada.drop_duplicates()

In [76]:
res_duplicate_ca.shape

(1635, 6)

In [77]:
res_duplicate_ca.to_csv("data_scientist_in_canada.csv",index=False)

## Scraper les offres en informatique BI avec au plus 1000 offres par ville:

### USA:

In [6]:
nb_resu_par_ville = 30
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set[:500]
URL="https://www.indeed.com/jobs?q=informatica"
t1=time.time()
res_info=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

nb_boocle:10278
len job_post=6
['Mount+Vernon', 'Informatica Developer with Healthcare Exp', "OVERVIEW: Our Client is Global leader in IT services, digital and business solutions that partners with its client to simplify, strengthen and transform their businesses. Our Client has been recognized by Brand Finance as one of the Big 4 Global IT Services Brands, is looking for a Informatica Developer with Healthcare Exp for project in NYC, NY, below is the detail requirement.\n\nResponsibilities:\n Works collaboratively with allstakeholders and apply expertise in Informatica Development, , Maintenance Design, develop, implement,troubleshoot, optimize, tune and assist and in validating ETL processes Debug or reverse engineer existingETL process for any enhancements or incidents or tuning them to performanceoptimization. Work with dev team lead andbusiness partners to clarify ETL requirements and business rules. Provides accurate level of effortestimates and delivery plans for projects and in

In [19]:
res_duplicate_BI_usa=res_info.drop_duplicates()
res_info.shape
res_duplicate_BI_usa.shape

(7380, 6)

In [22]:
res_duplicate_BI_usa.to_csv("developer_BI_usa.csv",index=False)
res_duplicate_BI_usa.shape

(7380, 6)

In [102]:
res_duplicate_BI_usa.iloc[15,:]

ville                                                      New+York
utituler_offre                       Applications Concierge Support
entreprise                                                   IPsoft
localisation                                                    NaN
resume            Well versed in ETL tools like SSIS/Informatica...
description       The cognitive revolution has arrived. Automati...
Name: 17, dtype: object

## UK BI:

In [ ]:
nb_resu_par_ville = 100
city_set = ["London","Birmingham","Leeds","Glasgow",
            "Sheffield","Bradford","Liverpool","Edinburgh",
           "Manchester","Bristol","Kirklees","Newcastle-upon-Tyne",
           "Kingston-upon-Hull","Bolton","Walsall","Plymouth",
           "Nottingham","Doncaster","Sunderland","Scottish+Borders",
           "Bassetlaw","Cheltenham","Erewash","East+Dunbartonshire",
           "Cambridgeshire","Norwich","Highland","Derby"]
URL="https://www.indeed.co.uk/jobs?q=informatica"
extension="https://www.indeed.co.uk"
res_actuariat_uk=scrap_indeed_offer(URL,city_set,nb_resu_par_ville,extension)

In [113]:
res_act_deduplicate_uk=res_actuariat_uk.drop_duplicates()
res_act_deduplicate_uk.shape

(234, 6)

In [109]:
res_act_deduplicate_uk.to_csv("developer_BI_uk.csv",index=False)

## Canada BI

In [110]:
nb_resu_par_ville = 100
city_set = ["Toronto","Montreal","Calgary","Ottawa",
            "Edmonton","Mississauga","Winnipeg","Vancouver",
           "Brampton","Hamilton","Surrey","Halifax",
            "Quebec+City","Markham","Gatineau","Oakville",
            "Delta","Terrebonne","Ajax","Lévis",
           "Oshawa","Windsor","Burnaby","Richmond"]
URL="https://ca.indeed.com/jobs?q=Informatica"
extension="https://ca.indeed.com"
res_canada=scrap_indeed_offer(URL,city_set,nb_resu_par_ville,extension)

nb_boocle:1744
len job_post=6
['Richmond', 'Data Architect - Visualization', 'ContractMatchBox Consulting Group is currently looking for a Data Architect for a 12-month full time contract position expected to start January 2019. This is a hands-on role, we’re looking for a strong resource in the Data Visualization space (Tableau and Power BI) and designing semantic layers to support Analytics (Views, data marts, DAT, others).\nThe Data Architect - Visualization will work under the managerial direction of the Analytics Program Manager and the Manager, Innovation and Enterprise Architecture. The Data Architect for Visualization will turn data into information, information into insight and insight into business decisions by applying advanced data wrangling, cleansing and visualization techniques and best practices to tell the stories behind the numbers and delivery of highly visual and interactive reports and dashboards.\nRequirements\nExperience with business intelligence methodologies i

In [111]:
res_dedup_BI_canada=res_canada.drop_duplicates()

In [114]:
res_dedup_BI_canada.shape

(815, 6)

## scrap actuariat job in USA:

In [3]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
URL="https://www.indeed.com/jobs?q=finance+actuarial"
t1=time.time()
res_FA=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_FA_usa=res_FA.drop_duplicates()
print(res_duplicate_FA_usa.shape)
res_duplicate_FA_usa.to_csv("Finance_actuariat_usa.csv",index=False)

nb_boocle:22122
len job_post=6
['Panama+City', 'Risk Management Insurance Analyst **Re-...', "$2,937 - $4,699 a monthThis is varied and highly responsible paraprofessional and specialized administrative work, technical in nature involving the coordination of risk management and safety programs including commercial insurance related and employee related programs. Work to be performed under the general supervision of the Assistant Risk Management Director.\n Essential Functions and Responsibilities:\nAssists RM Director and Assistant RM with all insurance priorities.\nAssists in the update/revision of the County's Risk Management Policies and Procedures.\nAssists with monitoring the issuance and receipt of certificates of insurance.\nAssists in organizing and collection of data and information for multi-line insurance claims.\nAssists RM Director with renewal marketing for all County Insurance policies, both P&C and Health Insurance products.\nMaintain an updated inventory of County owne

In [4]:
sum([ True if "actuar" in  k.lower()  or "insurance" in k.lower() or "cleam" in k.lower() else False for k in res_duplicate_FA_usa.utituler_offre])

4793

## Scrap digital marketing:

In [3]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
URL="https://www.indeed.com/jobs?q=Digital+Marketing"
t1=time.time()
res_DM=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_DM_usa=res_DM.drop_duplicates()
print(res_duplicate_DM_usa.shape)
res_duplicate_DM_usa.to_csv("Digital_marketing_usa.csv",index=False)

nb_boocle:37002
len job_post=6
['Panama+City', 'Mortgage Loan Officer', 'CommissionGrande Mortgages is seeking talented Mortgage Loan Officers who are looking to build a team in their area and work remotely across the state of Florida. Grande Mortgages is a division of Grande Companies that encompasses a Custom Home Builder, Real Estate Company, Credit Repair Company and Digital Marketing Company. If you would like to take on a leadership role and be a part of an extremely dynamic company, apply today!Job Type: CommissionLicense:FL Loan Originator License (Required)Sales cycle:Less than 90 daysTypical deal size:$1-5kOnboarding time:Less than 1 month', 'Grande Mortgages', 'Panama City, FL', 'Grande Mortgages is a division of Grande Companies that encompasses a Custom Home Builder, Real Estate Company, Credit Repair Company and Digital Marketing...']
Durée total réelle:45881.99751806259


/home/tantely/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


(28636, 6)


## Scrap nursary post

In [4]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
URL="https://www.indeed.com/jobs?q=Nurse"
t1=time.time()
res_Nurse=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_Nurse_usa=res_Nurse.drop_duplicates()
print(res_duplicate_Nurse_usa.shape)
res_duplicate_Nurse_usa.to_csv("Nurse_USA.csv",index=False)

nb_boocle:39546
len job_post=6
['Panama+City', 'RN Case Manager PRN', 'RN Case Manager PRN(Job Number: 00096-7303)\n\nWork Location: United States-Florida-Panama City-Gulf Coast Regional Medical Center\nSchedule: PRN/Per Diem\nJob Type: Case Management\n\n\nDescription\n\nGulf Coast Regional Medical Center is a 216-bed acute care hospital proudly serving Bay County and surrounding communities for 40 years. The hospital was recently named a Joint Commission Top Performer on Key Quality Measures® – heart failure, heart attack, pneumonia, surgical care, and perinatal care – for the fourth consecutive year and has achieved certifications for its stroke and wound care.\n\nLocated in Panama City, Florida, its team of more than 450 providers and 1,000 employees work to ensure patients receive the highest quality of care every day. With the area’s only designated Pediatric ER, four-bed Pediatric ICU and 28-bed NICU, the hospital is ready to care for our youngest residents. The hospital has the

## Scrap Surgeon post

In [5]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
URL="https://www.indeed.com/jobs?q=surgeon"
t1=time.time()
res_surgeon=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_surgeon_usa=res_surgeon.drop_duplicates()
print(res_duplicate_surgeon_usa.shape)
res_duplicate_surgeon_usa.to_csv("chirurgien_USA.csv",index=False)

nb_boocle:35835
len job_post=6
['Panama+City', 'Medical Technologist (PRN)', 'CommissionMedical Technologist (PRN)(Job Number: 00096-7385)\n\nWork Location: United States-Florida-Panama City-Gulf Coast Regional Medical Center\nSchedule: PRN/Per Diem\nJob Type: Laboratory\n\n\nDescription\n\nAt Gulf Coast Regional Medical Center, you’ll find a hospital that has earned the Joint Commission’s National Gold Seal of Approval for our spine, stroke and wound care programs.\n\nAdditionally, our cancer program is accredited with commendation by the American College of Surgeons Commission on Cancer.\n\nPosition Summary:\n\nThe Medical Technologist is responsible for various laboratory testing procedures in assigned areas as licensed.\n\nWork customarily involves analytical testing of blood and/or other human biological specimens. Depending on campus and department - various pre and post analytical work processes are included in the work duties customarily performed.\n\nQualifications:\n\nMust be

## Scrap Pharmacist job

In [3]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
URL="https://www.indeed.com/jobs?q=Pharmacist"
t1=time.time()
res_Pharmacist=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_Pharmacist_usa=res_Pharmacist.drop_duplicates()
print(res_duplicate_Pharmacist_usa.shape)
res_duplicate_Pharmacist_usa.to_csv("Pharmacist_USA.csv",index=False)

nb_boocle:30950
len job_post=6
['Panama+City', 'Clinical Pharmacist', "ContractJob Description:\nConcentric Methods is seeking an experienced Clinical\nPharmacist. This position is contingent upon contract\naward.\nJob Duty\nThe Clinical Pharmacist performs clinical and\neducational functions required in drug monitoring, with\nemphasis on appropriateness, safety and cost-effective\ndrug therapy.\nEssential Functions and Responsibilities\nAssess patient's response to drug therapy and planning\ndrug therapy based on physician established diagnoses\nOrder and evaluate laboratory tests necessary to\nevaluate drug therapy effects and outcomes\nInitiate, modify, or discontinue medications for ongoing\ntherapy of chronic disease states (e.g. hypertension,\nhyperlipidemia, anticoagulant, diabetes, asthma, refill\nclinics, etc.), in cooperation with the medical staff\nMonitor and manage pharmacotherapy requiring periodic\nadjustment due to specific or changing pharmacokinetic\ncharacteristics (

/home/tantely/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


(21094, 6)


 ## scrap dentist Job

In [4]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
URL="https://www.indeed.com/jobs?q=dentist"
t1=time.time()
res_dentist=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_dentist_usa=res_dentist.drop_duplicates()
print(res_duplicate_dentist_usa.shape)
res_duplicate_dentist_usa.to_csv("dentist_USA.csv",index=False)

nb_boocle:34873
len job_post=6
['Panama+City', 'Dentist (Weekends)', 'Part-timeGeneral Dentist needed to work Saturdays 8-5 Full time or Part Time availablePrefer Florida licensed but will consider experienced dentist licensed in another state.Competitive pay (DOE), All Full Time Benefits Include 13 paid holidays per year, paid time off (PTO), group medical, dental, vision & life benefits, plus an excellent retirement savings plan.Job Types: Full-time, Part-timeLicense:Dentist (Required)Benefits offered:Retirement benefits or accountsEmployee discountsOvertime often available:NoClinical Setting:General DentistryTypical start time:8AMTypical end time:5PM', 'PanCare of Florida, Inc.', 'Panama City, FL', '*General Dentist needed to work Saturdays 8-5  Full time or Part Time available* Prefer Florida licensed but will consider experienced dentist licensed in...']
Durée total réelle:45178.20732712746
(27495, 6)
